In [8]:
import numpy as np
import pandas as pd
import pdfplumber

In [9]:
from collections import Counter
import math
import string

In [10]:
def ekstrak_teks_dari_pdf(path):
    with pdfplumber.open(path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text(x_tolerance=2, y_tolerance=2)
    return text

In [11]:
# File yang sama
teks_1 = ekstrak_teks_dari_pdf("./jurnal/paper1.pdf")
print(teks_1[100:200])

lsevier.com/locate/displa
A unified architecture for super-resolution and segmentation of remote
✩
s


In [12]:
# Menghilangkan tanda baca
translator = str.maketrans('', '', string.punctuation)

# Remove punctuation
teks_1_clean = teks_1.translate(translator)
print(teks_1_clean)

Displays 84 2024 102800
Contents lists available at ScienceDirect
Displays
journal homepage wwwelseviercomlocatedispla
A unified architecture for superresolution and segmentation of remote
✩
sensing images based on similarity feature fusion
Lunqian Wanga Xinghua Wanga Weilin Liua Hao Dinga Bo Xiaa Zekai Zhangb
Jinglin Zhangbc∗  Sen Xud
aDepartment of Information Science and Engineering Linyi University Linyi 276000 China
bDepartment of Control Science and Engineering Shandong University Jinan 250061 China
cShandong Research Institute of Industrial Technology Jinan 250013 China
dDepartment of Information Science and Engineering YanCheng Institute of Technology Yancheng 224000 China
A R T I C L E I N F O A B S T R A C T
MSC The resolution of the image has an important impact on the accuracy of segmentation Integrating super
41A05 resolution SR techniques in the semantic segmentation of remote sensing images contributes to the
41A10 improvement of precision and accuracy especially when th

In [13]:
# Artikel rekomendasi dari science direct
teks_2 = ekstrak_teks_dari_pdf("./jurnal/paper2.pdf")
print(teks_2[100:200])

lsevier.com/locate/displa
✩
FSNet: A dual-domain network for few-shot image classification
∗
Xuewen 


In [14]:
# Artikel yang disitasi paper ini
teks_3 = ekstrak_teks_dari_pdf("./jurnal/paper3.pdf")
print(teks_3[:100])

Multi-Scale High-Resolution Vision Transformer for Semantic Segmentation
Jiaqi Gu2*, Hyoukjun Kwon1,


In [15]:
# Artikel dengan author yang sama (jurnal beda)
teks_4 = ekstrak_teks_dari_pdf("./jurnal/paper4.pdf")
print(teks_4[100:200])

e at ScienceDirect
Engineering Applications of Artificial Intelligence
journal homepage: www.elsevie


In [16]:
# Artikel yang menyitasi paper ini
teks_5 = ekstrak_teks_dari_pdf("./jurnal/paper5.pdf")
print(teks_5[100:200])

mepage: www.elsevier.com/locate/neucom
A multimodal fusion framework for semantic segmentation of re


In [17]:
def tokenize(text):
    return text.lower().split()

In [18]:
def ngrams(tokens, n):
    return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

In [19]:
def cosine_similarity(ngrams1, ngrams2):
    c1, c2 = Counter(ngrams1), Counter(ngrams2)
    all_keys = set(c1.keys()) | set(c2.keys())
    
    dot = sum(c1[k] * c2[k] for k in all_keys)
    norm1 = math.sqrt(sum(v**2 for v in c1.values()))
    norm2 = math.sqrt(sum(v**2 for v in c2.values()))
    
    return dot / (norm1 * norm2) if norm1 and norm2 else 0

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity as cosine_sim_sklearn

def cek_n_gram_all(text_1, text_2):
    tokens1 = tokenize(text_1)
    tokens2 = tokenize(text_2)

    hasil = {}
    for n in [1, 2, 3]:
        # Cosine dengan n-gram manual
        ngrams1 = ngrams(tokens1, n)
        ngrams2 = ngrams(tokens2, n)
        cosine = cosine_similarity(ngrams1, ngrams2)
        hasil[f"Cosine_n{n}"] = cosine

        # Jaccard
        set1, set2 = set(ngrams1), set(ngrams2)
        jaccard = len(set1 & set2) / len(set1 | set2) if set1 | set2 else 0
        hasil[f"Jaccard_n{n}"] = jaccard

        # TF-IDF + Cosine
        vectorizer = TfidfVectorizer(ngram_range=(n, n))
        tfidf = vectorizer.fit_transform([text_1, text_2])
        tfidf_cosine = cosine_sim_sklearn(tfidf[0:1], tfidf[1:2])[0][0]
        hasil[f"TFIDF_Cosine_n{n}"] = tfidf_cosine

    return hasil




In [ ]:
perbandingan = [
    ("Dua file yang sama", teks_1, teks_1),
    ("Artikel yang direkomendasikan oleh ScienceDirect", teks_1, teks_2),
    ("Artikel yang disitasi paper ini", teks_1, teks_3),
    ("Artikel dengan author yang sama", teks_1, teks_4),
    ("Artikel yang menyitasi paper ini", teks_1, teks_5),
]

hasil = []
for n in [1, 2, 3]:
    for label, t1, t2 in perbandingan:
        # Menghilangkan tanda baca
        # translator = str.maketrans('', '', string.punctuation)

        # t1 = t1.translate(translator)
        # t2 = t2.translate(translator)

        metrics = cek_n_gram_all(t1, t2)
        hasil.append({
            "n": n,
            "Perbandingan": label,
            "Cosine": metrics[f"Cosine_n{n}"],
            "Jaccard": metrics[f"Jaccard_n{n}"],
            "TFIDF_Cosine": metrics[f"TFIDF_Cosine_n{n}"]
        })

df = pd.DataFrame(hasil)

# Pastikan urut sesuai n lalu perbandingan
df = df.sort_values(by=["n"]).reset_index(drop=True)

print(df)


    n                                      Perbandingan    Cosine   Jaccard  \
0   1                                Dua file yang sama  1.000000  1.000000   
1   1  Artikel yang direkomendasikan oleh ScienceDirect  0.886931  0.218020   
2   1                   Artikel yang disitasi paper ini  0.794914  0.188248   
3   1                   Artikel dengan author yang sama  0.892828  0.285790   
4   1                  Artikel yang menyitasi paper ini  0.895501  0.236964   
5   2                                Dua file yang sama  1.000000  1.000000   
6   2  Artikel yang direkomendasikan oleh ScienceDirect  0.404677  0.058105   
7   2                   Artikel yang disitasi paper ini  0.223014  0.037941   
8   2                   Artikel dengan author yang sama  0.401472  0.091966   
9   2                  Artikel yang menyitasi paper ini  0.463394  0.070110   
10  3                                Dua file yang sama  1.000000  1.000000   
11  3  Artikel yang direkomendasikan oleh ScienceDir

In [22]:
df.head()

,n,Perbandingan,Cosine,Jaccard,TFIDF_Cosine
0,1,Dua file yang sama,1.000000,1.000000,1.000000
1,1,Artikel yang direkomendasikan oleh ScienceDirect,0.886931,0.218020,0.861000
2,1,Artikel yang disitasi paper ini,0.794914,0.188248,0.767788
3,1,Artikel dengan author yang sama,0.892828,0.285790,0.896744
4,1,Artikel yang menyitasi paper ini,0.895501,0.236964,0.881870


In [23]:
latex_table = df.to_latex(index=False, float_format="%.4f")

print(latex_table)

\begin{tabular}{rlrrr}
\toprule
n & Perbandingan & Cosine & Jaccard & TFIDF_Cosine \\
\midrule
1 & Dua file yang sama & 1.0000 & 1.0000 & 1.0000 \\
1 & Artikel yang direkomendasikan oleh ScienceDirect & 0.8869 & 0.2180 & 0.8610 \\
1 & Artikel yang disitasi paper ini & 0.7949 & 0.1882 & 0.7678 \\
1 & Artikel dengan author yang sama & 0.8928 & 0.2858 & 0.8967 \\
1 & Artikel yang menyitasi paper ini & 0.8955 & 0.2370 & 0.8819 \\
2 & Dua file yang sama & 1.0000 & 1.0000 & 1.0000 \\
2 & Artikel yang direkomendasikan oleh ScienceDirect & 0.4047 & 0.0581 & 0.2791 \\
2 & Artikel yang disitasi paper ini & 0.2230 & 0.0379 & 0.1436 \\
2 & Artikel dengan author yang sama & 0.4015 & 0.0920 & 0.3412 \\
2 & Artikel yang menyitasi paper ini & 0.4634 & 0.0701 & 0.3577 \\
3 & Dua file yang sama & 1.0000 & 1.0000 & 1.0000 \\
3 & Artikel yang direkomendasikan oleh ScienceDirect & 0.0817 & 0.0156 & 0.0460 \\
3 & Artikel yang disitasi paper ini & 0.0251 & 0.0072 & 0.0133 \\
3 & Artikel dengan author yang sa